<a href="https://colab.research.google.com/github/zhengyanggao02/zhengyanggao_DTSC3020Fall2025/blob/main/PartB_small_map.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import networkx as nx
import random

random.seed(1000)

# 1. small graph edges
edges_small = [
    (0, 2, 11),
    (0, 4, 7),
    (2, 1, 4),
    (2, 3, 9),
    (3, 4, 3),
]

F = 0.8
New_path_n = 3

Trips_per_sec = 100
Sim_seconds   = 3600
Total_trips   = Trips_per_sec * Sim_seconds

In [2]:
# 2. build the graph
G = nx.Graph()
for u, v, w in edges_small:
    G.add_edge(u, v, length=w)

node_list = [0, 1, 2, 3, 4]

In [3]:
# 3. simulate trips per spec: 100/sec for 3600 sec
pair_flow = {}

for _sec in range(Sim_seconds):
    for _ in range(Trips_per_sec):
        s, t = random.sample(node_list, 2)
        _ = nx.dijkstra_path(G, s, t, weight='length')
        if (s, t) not in pair_flow:
            pair_flow[(s, t)] = 0
        pair_flow[(s, t)] += 1

In [4]:
# 4. all-pairs shortest path distances
def all_pairs_dist(graph):
    dist = {}
    for src, d in nx.all_pairs_dijkstra_path_length(graph, weight='length'):
        for dst, val in d.items():
            dist[(src, dst)] = val
    return dist

dist_dict = all_pairs_dist(G)

In [5]:
# 5. benefit for possible new roads (only pairs without a direct edge)
benefit_dict = {}
existing_edges = set((min(u, v), max(u, v)) for u, v in G.edges())

for i in range(len(node_list)):
    for j in range(i + 1, len(node_list)):
        if (i, j) in existing_edges:
            continue
        sp = dist_dict[(i, j)]
        flow = pair_flow.get((i, j), 0) + pair_flow.get((j, i), 0)
        if flow <= 0:
            continue
        new_len = F * sp
        benefit_dict[(i, j)] = (sp - new_len) * flow

In [6]:
# 6. greedy pick of new roads
chosen_paths = []
for _ in range(New_path_n):
    if not benefit_dict:
        break
    (x, y), _ = max(benefit_dict.items(), key=lambda kv: kv[1])
    sp = dist_dict[(x, y)]
    new_L = F * sp
    G.add_edge(x, y, length=new_L)
    chosen_paths.append((x, y, new_L))
    dist_dict = all_pairs_dist(G)
    benefit_dict = {}

In [7]:
# 7. average shortest path length (before/after)
def avg_shortest(graph):
    d = all_pairs_dist(graph)
    total, cnt = 0.0, 0
    for a in node_list:
        for b in node_list:
            if a == b:
                continue
            total += d[(a, b)]
            cnt += 1
    return total / cnt

G_before = nx.Graph()
for u, v, w in edges_small:
    G_before.add_edge(u, v, length=w)

G_after = G_before.copy()
for x, y, L in chosen_paths:
    G_after.add_edge(x, y, length=L)

avg_before = avg_shortest(G_before)
avg_after  = avg_shortest(G_after)

In [8]:
# 8. print
print("R3 — suggested new roads:")
if chosen_paths:
    for x, y, L in chosen_paths:
        print(f"  - ({x}, {y}), length = {L:.2f}")
else:
    print("  - none")

print("\nR4 — avg shortest path length:")
print(f"  - before: {avg_before:.4f}")
print(f"  - after : {avg_after:.4f}")
print(f"  - improve: {avg_before - avg_after:.4f}")

print("\nR5 — conclusion:")
if chosen_paths:
    print("  - after adding new road(s), average distance went down a bit.")
else:
    print("  - no useful new road found, maybe tweak F or increase trips.")

R3 — suggested new roads:
  - (1, 4), length = 12.80

R4 — avg shortest path length:
  - before: 10.0000
  - after : 9.6800
  - improve: 0.3200

R5 — conclusion:
  - after adding new road(s), average distance went down a bit.
